# Prompt templates and variables

This notebook introduces how to create and use prompt templates with variables to enhance interactions with AI language models. Templates make our prompts easier to manage, more adaptable, and reusable across different use cases. We will explore how to generate dynamic prompts for OpenAI's language models using two approaches:
- LangChain's `PromptTemplate` for simple, structured inputs.
- Custom Jinja2-based templates for advanced use cases like conditionals and loops. Using Jinja2 templating engine, we will learn to build flexible, reusable prompt structures that support dynamic content. 

Both methods allow you to define prompts that can be reused and dynamically adapted based on input. This approach makes it easier to manage, scale, and maintain complex applications involving AI.

In [1]:
import os
import openai
import openai
from openai import OpenAI
from dotenv import load_dotenv
from jinja2 import Template
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Load environment variables from .env file
load_dotenv()

# Set OpenAI API key
openai.api_key = os.getenv('OPENAI_API_KEY')

#### Initialize the language model
We now initialize the `ChatOpenAI` model from LangChain. Here, we use `gpt-4o-mini`, a lightweight and responsive model suitable for interactive use cases.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

### Simple template with one variable
We start with a basic example: a template that asks the model to explain a topic. The variable `{topic}` can be replaced dynamically.

In [3]:
# Creating a simple template with one variable
simple_template = PromptTemplate(
    template="Provide a brief explanation of {topic}.",
    input_variables=["topic"]
)

# Using the simple template
print("Simple Template Result:")
# Combine the template and the model into a prompt chain
chain = simple_template | llm
# Use the prompt chain by passing a topic
print(chain.invoke({"topic": "photosynthesis"}).content)

Simple Template Result:
Photosynthesis is the biological process by which green plants, algae, and some bacteria convert light energy, usually from the sun, into chemical energy stored in glucose. This process primarily occurs in the chloroplasts of plant cells, where chlorophyll—a green pigment—absorbs light. 

During photosynthesis, carbon dioxide (from the air) and water (from the soil) are combined using light energy to produce glucose and oxygen. The overall chemical equation for photosynthesis can be summarized as:

\[ 6CO_2 + 6H_2O + light \ energy \rightarrow C_6H_{12}O_6 + 6O_2 \]

In this equation, \( CO_2 \) represents carbon dioxide, \( H_2O \) represents water, \( C_6H_{12}O_6 \) represents glucose, and \( O_2 \) represents oxygen. Photosynthesis is crucial for life on Earth as it provides the oxygen we breathe and serves as the foundation of the food chain.


- We define a `PromptTemplate` using LangChain with one placeholder. It expects a single variable called `topic`.
- The `|` operator (pipe) creates a chain that passes the prompt result directly to the model (`llm`) using LangChain’s syntax.
- `chain.invoke()` feeds the topic into the template, constructs the prompt, sends it to the model, and returns the completion. When invoking, we supply a value for `topic` ("photosynthesis").


### Template with multiple variables
Now let’s make things a bit more dynamic with multiple variables. This is useful when context needs to be adapted to a specific audience.

In [4]:
# Creating a more complex template with multiple variables
complex_template = PromptTemplate(
    template="Explain the concept of {concept} in the field of {field} to a {audience} audience, concisely.",
    input_variables=["concept", "field", "audience"]
)

# Using the complex template
print("Complex Template Result:")
# Combine the template and the model into a prompt chain
chain = complex_template | llm
# Invoke the chain with multiple input values
print(chain.invoke({"concept": "neural networks", "field": "artificial intelligence", "audience": "beginner"}).content)

Complex Template Result:
Neural networks are a key technology in artificial intelligence that are inspired by how the human brain works. They consist of interconnected groups of nodes, or "neurons," that process information in layers. 

Here's a simple breakdown:

1. **Structure**: A neural network has an input layer, one or more hidden layers, and an output layer. Each layer contains neurons that receive inputs, process them, and pass on their outputs to the next layer.

2. **Learning**: Neural networks learn by adjusting the connections (or weights) between neurons based on examples they are trained on. This training process helps them recognize patterns in data.

3. **Function**: Once trained, a neural network can make predictions or decisions based on new input. For example, it can identify objects in images, translate languages, or even generate music.

Overall, neural networks are powerful tools for solving complex problems in various fields, including image recognition, natural 

- The template now includes `{concept}`, `{field}`, and `{audience}`.
- We pass a dictionary to `invoke()` to fill in all required fields.
- The model now has a clearer context (what, where, and for whom), which improves the quality and relevance of the generated output.


### Template for list processing
Templates can also process structured input like lists. Here we pass a comma-separated string of items and ask the model to group them.

In [5]:
# Template for categorizing a list of items
list_template = PromptTemplate(
    template="Categorize these items into groups: {items}. Provide the categories and the items in each category.",
    input_variables=["items"]
)

print("List Template Result:")
# Combine the template and the model
chain = list_template | llm
# Input: a comma-separated list of mixed items
items = "apple, banana, carrot, hammer, screwdriver, pliers, novel, textbook, magazine"
print(chain.invoke({"items": items}).content)

List Template Result:
Here are the categories and the items grouped accordingly:

### Fruits
- Apple
- Banana

### Vegetables
- Carrot

### Tools
- Hammer
- Screwdriver
- Pliers

### Literature
- Novel
- Textbook
- Magazine


- This template receives a list (in string format) and prompts the model to organize it into categories. The prompt remains reusable — only the `items` input changes.
- The model parses the list, identifies patterns, and classifies them into logical groupings such as fruits, tools, books, etc. This is helpful for tasks like classification or organization.


### Template with dynamic instructions
Templates can also be used to inject instructions, context, and constraints dynamically into a prompt.

In [6]:
# Define a template with multiple labeled instruction sections
dynamic_instruction_template = PromptTemplate(
    template=(
        "Task: {task}\n"
        "Context: {context}\n"
        "Constraints: {constraints}\n\n"
        "Please provide a solution that addresses the task, considers the context, and adheres to the constraints."
    ),
    input_variables=["task", "context", "constraints"]
)

print("Dynamic Instruction Template Result:")
# Chain the template with the model
chain = dynamic_instruction_template | llm

# Invoke with detailed structured input
print(chain.invoke({
    "task": "Design a logo for a tech startup",
    "context": "The startup focuses on AI-driven healthcare solutions",
    "constraints": "Must use blue and green colors, and should be simple enough to be recognizable when small"
}).content)

Dynamic Instruction Template Result:
Designing a logo for a tech startup focused on AI-driven healthcare solutions involves several key elements to ensure it is effective and memorable. Here’s a conceptual solution:

### Logo Concept:

1. **Shape and Symbolism**:
   - **Caduceus + Circuit**: Combine the symbol of a caduceus (representing healthcare) with circuit-like lines or nodes to evoke technology and AI. The caduceus can be stylized to be more abstract, ensuring that it remains recognizable at small sizes.
   - **Brain Icon**: Integrate a simple representation of a brain with digital elements (like pixels or circuitry) to symbolize AI. This can be placed next to or within the caduceus shape.

2. **Color Palette**:
   - **Blue**: Use a calming, professional shade of blue for the healthcare aspect, symbolizing trust and reliability. 
   - **Green**: Incorporate a vibrant green to represent growth, health, and innovation. This can be used as an accent or in the circuitry part of the 

- The template ensures that input is structured logically, making it easier for the model to follow.
- This pattern is excellent for generating consistent responses in design, planning, or decision-making prompts.


## Jinja-based templates for advanced logic
The LangChain `PromptTemplate` doesn’t support `if/else` logic or loops. For that, we will define our own template system using Jinja2 with a custom `PromptTemplate` class.

In [7]:
# Create OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Function to call OpenAI API directly
def get_completion(prompt, model="gpt-4o-mini-2024-07-18"):
    """
    Get a completion from the OpenAI API

    Args:
        prompt (str): The prompt to send to the API
        model (str): The model to use for the completion
    Returns:
        str: The generated completion text from the model
    """
    # Wrap the prompt in the format expected by the Chat API
    messages = [{"role": "user", "content": prompt}]
    
    response = client.chat.completions.create(
        model=model, # Specify which model to use
        messages=messages, # Provide the conversation as a list of messages
        temperature=0, # Set deterministic behavior (lower temperature = less randomness)
    )
    # Extract the content of the model's reply from the response object
    return response.choices[0].message.content

This function is a reusable utility that handles sending prompts to OpenAI’s Chat API and returns the AI's response.
- The function takes a string `prompt` and an optional `model` parameter (defaults to GPT-4o-mini)
- OpenAI’s chat models expect the input to be in a list of message dictionaries, where each dictionary includes:
     - `role`: Identifies the speaker (`"user"`, `"assistant"`, or `"system"`). In this case, the prompt is wrapped in a message with the `"user"` role.
     - `content`: The actual text to be interpreted by the model.
- The function uses the `OpenAI` client object to call `client.chat.completions.create(...)` to send the message(s) to the selected model. (OpenAI client serves as the primary interface to interact with OpenAI’s API)
- The API returns a structured response object. `response.choices[0].message.content` accesses the first result returned (since multiple choices can be returned if `n > 1`) and retrieves the actual reply from the assistant.
- Finally, the function returns the plain text of the AI's response for easy use in further code or display.

#### Custom `PromptTemplate` class using Jinja2
This class provides a simple wrapper around the powerful Jinja2 templating engine, making it easier to create reusable prompt templates with dynamic content and allowing us to use advanced logic like conditionals (`{% if %}`) and loops (`{% for %}`).

In [8]:
# Custom PromptTemplate using Jinja2
class PromptTemplate:
    ''' A class to represent a template for generating prompts with variables
    Attributes:
        template (str): The template string with variables
        input_variables (list): A list of the variable names in the template
    '''
    def __init__(self, template, input_variables):
        # Compile the Jinja2 template from the provided string
        self.template = Template(template)  # Initialize Jinja2 template
        self.input_variables = input_variables # Track which variables are required for this template
    
    def format(self, **kwargs):
        # Fill in the template using keyword arguments (kwargs) and return the rendered string
        return self.template.render(**kwargs)  # Render with passed variables

- `template`: A string written using Jinja2 syntax. It can include placeholders like `{{ name }}` or `{% if profession %}`.
- `Template(template)`: This turns the raw string into a Jinja2 Template object, which can later be rendered with data.
- `input_variables`: This is just metadata—a list of variable names that the template expects. This isn’t used directly by Jinja2, but it's useful for validation or documentation.
- The `format` method returns the fully rendered prompt string, ready to be sent to the language model.
- `**kwargs`: Accepts any number of keyword arguments representing the variables to fill into the template.
- `self.template.render(**kwargs)`: Jinja2 uses this method to substitute the placeholders in the template with actual values.

### Jinja-based conditional template
Now we create a prompt that conditionally includes a profession if it exists.

In [9]:
# Template with conditional logic using Jinja2
conditional_template = PromptTemplate(
    template="My name is {{ name }} and I am {{ age }} years old. "
             "{% if profession %}I work as a {{ profession }}.{% else %}I am currently not employed.{% endif %} "
             "Can you give me career advice based on this information? answer concisely.",
    input_variables=["name", "age", "profession"]
)

# Case: with profession
print("Conditional Template Result (with profession):")
prompt = conditional_template.format(name="Alex", age="28", profession="software developer")
print(get_completion(prompt))

# Case: without profession
print("\nConditional Template Result (without profession):")
prompt = conditional_template.format(name="Sam", age="22", profession="")
print(get_completion(prompt))

Conditional Template Result (with profession):
Sure, Alex! Here are some career tips for you as a software developer:

1. **Continuous Learning**: Stay updated with the latest technologies and programming languages. Consider online courses or certifications in areas like cloud computing, AI, or cybersecurity.

2. **Networking**: Attend industry meetups, conferences, and online forums to connect with other professionals. This can lead to job opportunities and collaborations.

3. **Build a Portfolio**: Work on personal or open-source projects to showcase your skills. A strong portfolio can set you apart in job applications.

4. **Soft Skills**: Develop communication and teamwork skills. These are crucial for collaborating with colleagues and stakeholders.

5. **Explore Specializations**: Consider specializing in areas like front-end, back-end, DevOps, or data science to enhance your marketability.

6. **Seek Feedback**: Regularly ask for feedback from peers and mentors to improve your co

- Jinja allows `if-else` conditions inside the prompt. `{% if profession %}...{% else %}...{% endif %}` checks if a profession is passed.
- This avoids redundant prompts and makes responses more natural and context-aware.
- The same template is used for different user states—employed or not.

### Jinja-based list formatting with loop
Using a `for` loop inside a template to iterate through items and format them nicely for the model.

In [10]:
# Template with a formatted list
list_format_template = PromptTemplate(
    template="Analyze the following list of items:\n"
             "{% for item in items.split(',') %}"
             "- {{ item.strip() }}\n"
             "{% endfor %}"
             "\nProvide a summary of the list and suggest any patterns or groupings.",
    input_variables=["items"]
)

# Using the formatted list template
print("Formatted List Template Result:")
prompt = list_format_template.format(
    items="Python, JavaScript, HTML, CSS, React, Django, Flask, Node.js"
)
print(get_completion(prompt))

Formatted List Template Result:
The list of items you provided consists of programming languages, frameworks, and technologies commonly used in web development. Here's a summary and analysis of the items:

### Summary of the List:
1. **Programming Languages:**
   - **Python**: A versatile, high-level programming language known for its readability and wide range of applications, including web development, data analysis, artificial intelligence, and more.
   - **JavaScript**: A core web technology that enables interactive web pages and is essential for front-end development. It can also be used on the server side with environments like Node.js.

2. **Markup and Styling Languages:**
   - **HTML (HyperText Markup Language)**: The standard markup language for creating web pages. It structures the content on the web.
   - **CSS (Cascading Style Sheets)**: A stylesheet language used for describing the presentation of a document written in HTML. It controls layout, colors, fonts, and overall v

- The template uses a loop to format each item as a bullet point.
- We split a string into individual items using `.split(',')`.
- We could easily adapt this for checklists, logs, item summaries, and more.